In [1]:
import pandas as pd
import numpy as np
from create_sCASP_files import *



# 01

In [728]:
norm = 'L2'

In [792]:
model = 'MC3G'
#model = 'C3G'

scasp_path = './s(CASP)_'+model+'/'
#scasp_path = './s(CASP)'

In [793]:
model

'MC3G'

In [794]:

train_path = './Data/train_fold_1.csv'
test_path = './Data/test_fold_1.csv'

# Index of the source instance in the test dataset. We find the counterfactual of this instance 
ref_index=1

In [795]:
k = 20

In [796]:
df_org = pd.read_csv(train_path)

df_org[:6]

,Age,Workclass,Fnlwgt,Education_num,Marital_Status,Occupation,Relationship,Race,Sex,Capital_gain,Hours_per_week,Native_country,Label
0,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,13,United-States,<=50K
1,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,40,Cuba,<=50K
2,37,Private,284582,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,40,United-States,<=50K
3,49,Private,160187,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,16,Jamaica,<=50K
4,52,Self-emp-not-inc,209642,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,45,United-States,>50K
5,31,Private,45781,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,50,United-States,>50K


In [797]:
### Select the features that contribute to the prediction
Feature_Columns = ['Marital_Status','Capital_gain','Education_num','Relationship','Sex']
print(Feature_Columns)

['Marital_Status', 'Capital_gain', 'Education_num', 'Relationship', 'Sex']


In [798]:
df_ref = pd.read_csv(test_path)
df_fact = df_ref.copy()[Feature_Columns]

In [799]:
df_fact[df_fact['Marital_Status']=='Divorced']

,Marital_Status,Capital_gain,Education_num,Relationship,Sex
1,Divorced,0,9,Not-in-family,Male
6,Divorced,0,9,Not-in-family,Male
21,Divorced,0,12,Unmarried,Female
35,Divorced,0,13,Not-in-family,Female
37,Divorced,0,10,Not-in-family,Female
...,...,...,...,...,...
6496,Divorced,0,8,Other-relative,Male
6499,Divorced,0,5,Not-in-family,Male
6502,Divorced,0,6,Not-in-family,Female
6505,Divorced,0,11,Unmarried,Female


In [800]:
df_fact.iloc[1]

Marital_Status         Divorced
Capital_gain                  0
Education_num                 9
Relationship      Not-in-family
Sex                        Male
Name: 1, dtype: object

In [801]:
df_org_1 = df_org[Feature_Columns].copy()
df_org_1

,Marital_Status,Capital_gain,Education_num,Relationship,Sex
0,Married-civ-spouse,0,13,Husband,Male
1,Married-civ-spouse,0,13,Wife,Female
2,Married-civ-spouse,0,14,Wife,Female
3,Married-spouse-absent,0,5,Not-in-family,Female
4,Married-civ-spouse,0,9,Husband,Male
...,...,...,...,...,...
26043,Never-married,0,10,Not-in-family,Male
26044,Married-civ-spouse,0,9,Husband,Male
26045,Widowed,0,9,Unmarried,Female
26046,Never-married,0,9,Own-child,Male


In [802]:
df_fact.iloc[0]

Marital_Status    Never-married
Capital_gain               2174
Education_num                13
Relationship      Not-in-family
Sex                        Male
Name: 0, dtype: object

In [803]:
# Obtain a dictionary with domain information about features in 'Feature_Columns
info_dict = build_feature_info(df_org_1, Feature_Columns)
#info_dict = build_feature_info_1(df_org_1, Feature_Columns, df_ref.iloc[ref_index], 10)
print(info_dict)

{'Marital_Status': ['Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Divorced', 'Separated', 'Married-AF-spouse', 'Widowed'], 'Capital_gain': {'range': (-4356, 99999), 'distinct_values': [-4356, -3900, -3770, -3683, -3004, -2824, -2754, -2603, -2559, -2547, -2489, -2472, -2457, -2444, -2415, -2392, -2377, -2352, -2339, -2282, -2267, -2258, -2246, -2238, -2231, -2206, -2205, -2179, -2174, -2163, -2149, -2129, -2080, -2057, -2051, -2042, -2002, -2001, -1980, -1977, -1974, -1902, -1887, -1876, -1848, -1844, -1825, -1816, -1762, -1755, -1741, -1740, -1726, -1721, -1719, -1672, -1669, -1668, -1651, -1648, -1628, -1617, -1602, -1594, -1590, -1579, -1573, -1564, -1539, -1504, -1485, -1408, -1380, -1340, -1258, -1138, -1092, -974, -880, -810, -653, -625, -419, -323, -213, -155, 0, 114, 401, 594, 914, 991, 1055, 1086, 1111, 1151, 1173, 1409, 1424, 1471, 1506, 1639, 1797, 1831, 1848, 2009, 2036, 2050, 2062, 2105, 2174, 2176, 2202, 2228, 2290, 2329, 2346, 2354, 2387, 2407, 2414, 2

In [804]:
# Write Domain information to a s(CASP) file for the s(CASP) code
write_domains(info_dict, scasp_path+"Domain_7.pl")
# Write Property information to a s(CASP) file for the s(CASP) code
write_properties(Feature_Columns, scasp_path+"Property_7.pl")

Domain facts written to ./s(CASP)_MC3G/Domain_7.pl
Wrote all feature lines to ./s(CASP)_MC3G/Property_7.pl


In [805]:
# Obtain the information of the source sample that we need a counterfactual of
# Store this information in a s(CASP) file as we need to create the counterfactual


# Print the source to view its attributes
print(df_fact.iloc[ref_index])
# Write this sample to the s(CASP) file
write_original_sample_auto(df_fact, ref_index, scasp_path+"Original_sample.pl", Feature_Columns, 'original_sample')

Marital_Status         Divorced
Capital_gain                  0
Education_num                 9
Relationship      Not-in-family
Sex                        Male
Name: 1, dtype: object
Wrote predicate to ./s(CASP)_MC3G/Original_sample.pl


In [806]:
df_fact[:6]

,Marital_Status,Capital_gain,Education_num,Relationship,Sex
0,Never-married,2174,13,Not-in-family,Male
1,Divorced,0,9,Not-in-family,Male
2,Married-civ-spouse,0,7,Husband,Male
3,Married-civ-spouse,0,13,Husband,Male
4,Never-married,0,12,Not-in-family,Male
5,Never-married,0,9,Own-child,Male


In [807]:
# Obtain the information of the mutability of the requied features
# 'Z'- no constraint
# 0 - Stay constant i.e. immutable
# 1 - change (for categorical features)/ increase (for numeric features)
# -1 - decrease (for numeric features)

mutability_constraints = ['Z','Z','Z', 'Z','Z']

write_mutability_predicate(
    mutability_list=mutability_constraints,
    output_file=scasp_path+"Mutability.pl",
    predicate_name="mutability")

Wrote predicate to /Users/spam/Documents/UTD/PHD/ALPS/Spring_25/NeSY_Late_Deadline/Check_Datasets_adult_5/s(CASP)_MC3G/Mutability.pl


In [808]:
# Obtain the information of the mutability of the requied features
# 'W': no weight - undesired
# number- given weight o the number
weight_constraints = [1,1,1,1,1]

write_weight_predicate(
    weight_list=weight_constraints,
    output_file=scasp_path+"Weight.pl",
    predicate_name="weight"
)


Wrote predicate to /Users/spam/Documents/UTD/PHD/ALPS/Spring_25/NeSY_Late_Deadline/Check_Datasets_adult_5/s(CASP)_MC3G/Weight.pl


# 02

In [809]:
from sCASP_subprocess import *



In [810]:
scasp_path

'./s(CASP)_MC3G/'

In [811]:
scasp_opt_file = "cf_7.txt"
scasp_file = "test_2.pl"


run_sCASP_script(scasp_opt_file, scasp_file,scasp_path+'/')

Output:
 % QUERY:?- original_sample(F_Marital_Status,F_Relationship,F_Sex,F_Capital_gain,F_Education_num),mutability(Z1,Z2,Z3,Z4,Z5),weight(W1,W2,W3,W4,W5),refined(original(F_Marital_Status,F_Relationship,F_Sex,F_Capital_gain,F_Education_num),id(Z1,Z2,Z3,Z4,Z5),counterfactual(Marital_Status,Capital_gain,Education_num,Relationship,Sex),weights(W1,W2,W3,W4,W5),X,cost(O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex)).

	ANSWER:	1 (in 3.096 ms)

MODEL:
{  }
BINDINGS: 
F_Marital_Status = Divorced 
F_Relationship = 0 
F_Sex = 9 
F_Capital_gain = Not-in-family 
F_Education_num = Male 
Z1 = 1 
Z2 = 1 
Z3 = 0 
Z4 = 1 
Z5 = 0 
W1 = 1 
W2 = 1 
W3 = 1 
W4 = 1 
W5 = 1 
Marital_Status = Married-civ-spouse 
Capital_gain #> 5013,Capital_gain #=< 99999 
Education_num = 9 
Relationship = Husband 
Sex = Male 
O_Marital_Status = 0 
O_Capital_gain = 1 
O_Education_num = 0 
O_Relationship = 0 
O_Sex = 0


	ANSWER:	2 (in 0.701 ms)

MODEL:
{  }
BINDINGS: 
F_Marital_Status = Divorced 
F_Re

In [812]:
# We create and save out counterfactual dataframe 
from create_cfs import *
# Counterfactual dataframe anme
cf_df_name = model+'_adult_counterfactuals.csv'

# Create the DataFrame of counterfactuals
instances = create_dataframe_from_textfile(scasp_path+scasp_opt_file)

In [813]:
instances

['F_Marital_Status = Divorced \nF_Relationship = 0 \nF_Sex = 9 \nF_Capital_gain = Not-in-family \nF_Education_num = Male \nZ1 = 1 \nZ2 = 1 \nZ3 = 0 \nZ4 = 1 \nZ5 = 0 \nW1 = 1 \nW2 = 1 \nW3 = 1 \nW4 = 1 \nW5 = 1 \nMarital_Status = Married-civ-spouse \nCapital_gain #> 5013,Capital_gain #=< 99999 \nEducation_num = 9 \nRelationship = Husband \nSex = Male \nX = X \nO_Marital_Status = 0 \nO_Capital_gain = 1 \nO_Education_num = 0 \nO_Relationship = 0 \nO_Sex = 0\n',
 ' \nF_Marital_Status = Divorced \nF_Relationship = 0 \nF_Sex = 9 \nF_Capital_gain = Not-in-family \nF_Education_num = Male \nZ1 = 1 \nZ2 = 1 \nZ3 = 1 \nZ4 = 1 \nZ5 = 0 \nW1 = 1 \nW2 = 1 \nW3 = 1 \nW4 = 1 \nW5 = 1 \nMarital_Status = Married-civ-spouse \nCapital_gain #> 5013,Capital_gain #=< 99999 \nEducation_num #> 9,Education_num #=< 16 \nRelationship = Husband \nSex = Male \nX = X \nO_Marital_Status = 0 \nO_Capital_gain = 1 \nO_Education_num = 1 \nO_Relationship = 0 \nO_Sex = 0\n',
 ' \nF_Marital_Status = Divorced \nF_Relationsh

In [814]:
len(instances)

186

In [815]:
# Identifying the Features of Interest and the Features corresponding to the weight set by the s(CASP) file
# s(CASP) will set the weight of causally altered features to 0 as they have 0 cost
print("We know the feature columns are: ",Feature_Columns)
columns_of_interest = Feature_Columns.copy()
Output_Weight_Columns = []

for i in Feature_Columns:
    Output_Weight_Columns.append('O_'+i)
    columns_of_interest.append('O_'+i)

We know the feature columns are:  ['Marital_Status', 'Capital_gain', 'Education_num', 'Relationship', 'Sex']


In [816]:
# We add a column CF_id to identify what counterfactual of the s(CASP) file a particular instance in the counterfactual dataframe belongs to
sCASP_df_columns = columns_of_interest.copy()
sCASP_df_columns.append('CF_id')
sCASP_df_columns

['Marital_Status',
 'Capital_gain',
 'Education_num',
 'Relationship',
 'Sex',
 'O_Marital_Status',
 'O_Capital_gain',
 'O_Education_num',
 'O_Relationship',
 'O_Sex',
 'CF_id']

In [817]:
# We already have the domain information about the desired column
print("Desired Domain info: ",info_dict)

Desired Domain info:  {'Marital_Status': ['Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Divorced', 'Separated', 'Married-AF-spouse', 'Widowed'], 'Capital_gain': {'range': (-4356, 99999), 'distinct_values': [-4356, -3900, -3770, -3683, -3004, -2824, -2754, -2603, -2559, -2547, -2489, -2472, -2457, -2444, -2415, -2392, -2377, -2352, -2339, -2282, -2267, -2258, -2246, -2238, -2231, -2206, -2205, -2179, -2174, -2163, -2149, -2129, -2080, -2057, -2051, -2042, -2002, -2001, -1980, -1977, -1974, -1902, -1887, -1876, -1848, -1844, -1825, -1816, -1762, -1755, -1741, -1740, -1726, -1721, -1719, -1672, -1669, -1668, -1651, -1648, -1628, -1617, -1602, -1594, -1590, -1579, -1573, -1564, -1539, -1504, -1485, -1408, -1380, -1340, -1258, -1138, -1092, -974, -880, -810, -653, -625, -419, -323, -213, -155, 0, 114, 401, 594, 914, 991, 1055, 1086, 1111, 1151, 1173, 1409, 1424, 1471, 1506, 1639, 1797, 1831, 1848, 2009, 2036, 2050, 2062, 2105, 2174, 2176, 2202, 2228, 2290, 2329, 2346, 235

In [818]:
# We map the domain information to the respective columns for numeric features
# Setting the domains of the numeric features as s(CASP) returns ranges
candidate_map = {
    'Capital_gain': info_dict['Capital_gain']['distinct_values']#,
    ,'Education_num': info_dict['Education_num']['distinct_values']
}


In [819]:
len(candidate_map)
type(candidate_map)
candidate_map['Capital_gain']
candidate_map['Education_num']

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [820]:
df_org.dtypes

Age                int64
Workclass         object
Fnlwgt             int64
Education_num      int64
Marital_Status    object
Occupation        object
Relationship      object
Race              object
Sex               object
Capital_gain       int64
Hours_per_week     int64
Native_country    object
Label             object
dtype: object

In [821]:
candidate_map

{'Capital_gain': [-4356,
  -3900,
  -3770,
  -3683,
  -3004,
  -2824,
  -2754,
  -2603,
  -2559,
  -2547,
  -2489,
  -2472,
  -2457,
  -2444,
  -2415,
  -2392,
  -2377,
  -2352,
  -2339,
  -2282,
  -2267,
  -2258,
  -2246,
  -2238,
  -2231,
  -2206,
  -2205,
  -2179,
  -2174,
  -2163,
  -2149,
  -2129,
  -2080,
  -2057,
  -2051,
  -2042,
  -2002,
  -2001,
  -1980,
  -1977,
  -1974,
  -1902,
  -1887,
  -1876,
  -1848,
  -1844,
  -1825,
  -1816,
  -1762,
  -1755,
  -1741,
  -1740,
  -1726,
  -1721,
  -1719,
  -1672,
  -1669,
  -1668,
  -1651,
  -1648,
  -1628,
  -1617,
  -1602,
  -1594,
  -1590,
  -1579,
  -1573,
  -1564,
  -1539,
  -1504,
  -1485,
  -1408,
  -1380,
  -1340,
  -1258,
  -1138,
  -1092,
  -974,
  -880,
  -810,
  -653,
  -625,
  -419,
  -323,
  -213,
  -155,
  0,
  114,
  401,
  594,
  914,
  991,
  1055,
  1086,
  1111,
  1151,
  1173,
  1409,
  1424,
  1471,
  1506,
  1639,
  1797,
  1831,
  1848,
  2009,
  2036,
  2050,
  2062,
  2105,
  2174,
  2176,
  2202,
  2228,
  2

In [822]:
df_ref.iloc[ref_index]

Age                              38
Workclass                   Private
Fnlwgt                       215646
Education_num                     9
Marital_Status             Divorced
Occupation        Handlers-cleaners
Relationship          Not-in-family
Race                          White
Sex                            Male
Capital_gain                      0
Hours_per_week                   40
Native_country        United-States
Label                         <=50K
Name: 1, dtype: object

In [823]:
# We create and save out counterfactual dataframe 
from create_cfs import *

empty_df = pd.DataFrame(columns=sCASP_df_columns)

for i in range(len(instances)):
    parsed_dict = parse_constraints(instances[i])
    print(parsed_dict)
    #print("Hi")
    df_tmp = expand_to_dataframe_k(parsed_dict, columns_of_interest, candidate_map,df_ref.iloc[ref_index], k)
    # Add a column with the same value for all rows
    df_tmp['CF_id'] = i+1
    empty_df = pd.concat([empty_df, df_tmp], ignore_index=True)
    #break

# Remove the last row as there is a ; in cf.txt
empty_df.to_csv('./Counterfactuals/'+cf_df_name)


{'F_Marital_Status': 'Divorced', 'F_Relationship': '0', 'F_Sex': '9', 'F_Capital_gain': 'Not-in-family', 'F_Education_num': 'Male', 'Z1': '1', 'Z2': '1', 'Z3': '0', 'Z4': '1', 'Z5': '0', 'W1': '1', 'W2': '1', 'W3': '1', 'W4': '1', 'W5': '1', 'Marital_Status': 'Married-civ-spouse', 'Capital_gain': '>5013 & <=99999', 'Education_num': '9', 'Relationship': 'Husband', 'Sex': 'Male', 'X': 'X', 'O_Marital_Status': '0', 'O_Capital_gain': '1', 'O_Education_num': '0', 'O_Relationship': '0', 'O_Sex': '0'}
{'F_Marital_Status': 'Divorced', 'F_Relationship': '0', 'F_Sex': '9', 'F_Capital_gain': 'Not-in-family', 'F_Education_num': 'Male', 'Z1': '1', 'Z2': '1', 'Z3': '1', 'Z4': '1', 'Z5': '0', 'W1': '1', 'W2': '1', 'W3': '1', 'W4': '1', 'W5': '1', 'Marital_Status': 'Married-civ-spouse', 'Capital_gain': '>5013 & <=99999', 'Education_num': '>9 & <=16', 'Relationship': 'Husband', 'Sex': 'Male', 'X': 'X', 'O_Marital_Status': '0', 'O_Capital_gain': '1', 'O_Education_num': '1', 'O_Relationship': '0', 'O_Sex

In [824]:
k

20

In [825]:
df_tmp

,Marital_Status,Capital_gain,Education_num,Relationship,Sex,O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex,CF_id
0,Widowed,7298,8,Other-relative,Female,1,1,1,1,1,186
1,Widowed,7298,7,Other-relative,Female,1,1,1,1,1,186
2,Widowed,7298,6,Other-relative,Female,1,1,1,1,1,186
3,Widowed,7298,5,Other-relative,Female,1,1,1,1,1,186
4,Widowed,7298,4,Other-relative,Female,1,1,1,1,1,186
...,...,...,...,...,...,...,...,...,...,...,...
155,Widowed,20051,5,Other-relative,Female,1,1,1,1,1,186
156,Widowed,20051,4,Other-relative,Female,1,1,1,1,1,186
157,Widowed,20051,3,Other-relative,Female,1,1,1,1,1,186
158,Widowed,20051,2,Other-relative,Female,1,1,1,1,1,186


In [826]:
print(parsed_dict)


{'F_Marital_Status': 'Divorced', 'F_Relationship': '0', 'F_Sex': '9', 'F_Capital_gain': 'Not-in-family', 'F_Education_num': 'Male', 'Z1': '1', 'Z2': '1', 'Z3': '-1', 'Z4': '1', 'Z5': '1', 'W1': '1', 'W2': '1', 'W3': '1', 'W4': '1', 'W5': '1', 'Marital_Status': 'Widowed', 'Capital_gain': '>6849 & <=99999', 'Education_num': '>=1 & <9', 'Relationship': 'Other-relative', 'Sex': 'Female', 'X': 'X', 'O_Marital_Status': '1', 'O_Capital_gain': '1', 'O_Education_num': '1', 'O_Relationship': '1', 'O_Sex': '1'}


# 03

In [827]:
empty_df

,Marital_Status,Capital_gain,Education_num,Relationship,Sex,O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex,CF_id
0,Married-civ-spouse,5060,9,Husband,Male,0,1,0,0,0,1
1,Married-civ-spouse,5178,9,Husband,Male,0,1,0,0,0,1
2,Married-civ-spouse,5455,9,Husband,Male,0,1,0,0,0,1
3,Married-civ-spouse,5556,9,Husband,Male,0,1,0,0,0,1
4,Married-civ-spouse,5721,9,Husband,Male,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
18587,Widowed,20051,5,Other-relative,Female,1,1,1,1,1,186
18588,Widowed,20051,4,Other-relative,Female,1,1,1,1,1,186
18589,Widowed,20051,3,Other-relative,Female,1,1,1,1,1,186
18590,Widowed,20051,2,Other-relative,Female,1,1,1,1,1,186


In [828]:
from zscorescaler import *
from get_dist import *

In [829]:
df_cf = pd.read_csv('./Counterfactuals/'+cf_df_name)
lst_numeric_features = [ 'Capital_gain', 'Education_num']



k_smallest_cf_df_file_name = model+'_adult_'+str(k)+'_smallest_'+norm+'.csv'

df_ref

,Age,Workclass,Fnlwgt,Education_num,Marital_Status,Occupation,Relationship,Race,Sex,Capital_gain,Hours_per_week,Native_country,Label
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,40,United-States,<=50K
1,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,40,United-States,<=50K
2,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,40,United-States,<=50K
3,30,State-gov,141297,13,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,40,India,>50K
4,32,Private,205019,12,Never-married,Sales,Not-in-family,Black,Male,0,50,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6508,37,Private,179137,10,Divorced,Adm-clerical,Unmarried,White,Female,0,39,United-States,<=50K
6509,30,Private,345898,9,Never-married,Craft-repair,Not-in-family,Black,Male,0,46,United-States,<=50K
6510,45,State-gov,252208,9,Separated,Adm-clerical,Own-child,White,Female,0,40,United-States,<=50K
6511,43,Private,260761,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,40,Mexico,<=50K


In [830]:
df_ref
df_tmp_ref = df_ref.drop(columns = 'Label')
df_tmp_ref_2 = df_tmp_ref.copy()[Feature_Columns]
df_tmp_ref_2

,Marital_Status,Capital_gain,Education_num,Relationship,Sex
0,Never-married,2174,13,Not-in-family,Male
1,Divorced,0,9,Not-in-family,Male
2,Married-civ-spouse,0,7,Husband,Male
3,Married-civ-spouse,0,13,Husband,Male
4,Never-married,0,12,Not-in-family,Male
...,...,...,...,...,...
6508,Divorced,0,10,Unmarried,Female
6509,Never-married,0,9,Not-in-family,Male
6510,Separated,0,9,Own-child,Female
6511,Married-civ-spouse,0,9,Husband,Male


In [831]:
df_original_sample = df_tmp_ref_2.iloc[[ref_index]]
df_original_sample

,Marital_Status,Capital_gain,Education_num,Relationship,Sex
1,Divorced,0,9,Not-in-family,Male


In [832]:
# Perform Z-Score Scaling for train set
scaler = ZScoreScaler(features_to_scale=lst_numeric_features)
scaler.fit(df_org)

In [833]:
# Scale the original_sample
df_original_sample_2 = scaler.transform(df_original_sample)
df_original_sample_2

,Marital_Status,Capital_gain,Education_num,Relationship,Sex
1,Divorced,-0.132393,-0.418904,Not-in-family,Male


In [834]:
df_cf

,Unnamed: 0,Marital_Status,Capital_gain,Education_num,Relationship,Sex,O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex,CF_id
0,0,Married-civ-spouse,5060,9,Husband,Male,0,1,0,0,0,1
1,1,Married-civ-spouse,5178,9,Husband,Male,0,1,0,0,0,1
2,2,Married-civ-spouse,5455,9,Husband,Male,0,1,0,0,0,1
3,3,Married-civ-spouse,5556,9,Husband,Male,0,1,0,0,0,1
4,4,Married-civ-spouse,5721,9,Husband,Male,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18587,18587,Widowed,20051,5,Other-relative,Female,1,1,1,1,1,186
18588,18588,Widowed,20051,4,Other-relative,Female,1,1,1,1,1,186
18589,18589,Widowed,20051,3,Other-relative,Female,1,1,1,1,1,186
18590,18590,Widowed,20051,2,Other-relative,Female,1,1,1,1,1,186


In [835]:
# Scale the counterfactuals for distance measure
df_cf_2 = scaler.transform(df_cf)
df_cf_2

,Unnamed: 0,Marital_Status,Capital_gain,Education_num,Relationship,Sex,O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex,CF_id
0,0,Married-civ-spouse,0.568999,-0.418904,Husband,Male,0,1,0,0,0,1
1,1,Married-civ-spouse,0.585355,-0.418904,Husband,Male,0,1,0,0,0,1
2,2,Married-civ-spouse,0.623752,-0.418904,Husband,Male,0,1,0,0,0,1
3,3,Married-civ-spouse,0.637752,-0.418904,Husband,Male,0,1,0,0,0,1
4,4,Married-civ-spouse,0.660623,-0.418904,Husband,Male,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18587,18587,Widowed,2.646975,-1.972600,Other-relative,Female,1,1,1,1,1,186
18588,18588,Widowed,2.646975,-2.361024,Other-relative,Female,1,1,1,1,1,186
18589,18589,Widowed,2.646975,-2.749448,Other-relative,Female,1,1,1,1,1,186
18590,18590,Widowed,2.646975,-3.137872,Other-relative,Female,1,1,1,1,1,186


In [836]:
# Get the type of each feature and store in a list
feature_types = infer_feature_types(df_original_sample_2)
print(feature_types)


{'Marital_Status': 'categorical', 'Capital_gain': 'numeric', 'Education_num': 'numeric', 'Relationship': 'categorical', 'Sex': 'categorical'}


In [837]:
# reference_row: original sample that we compare the counterfactuals to for distance
reference_row = df_original_sample_2.iloc[0]
reference_row

Marital_Status         Divorced
Capital_gain          -0.132393
Education_num         -0.418904
Relationship      Not-in-family
Sex                        Male
Name: 1, dtype: object

In [838]:
df_cf_2.copy()

,Unnamed: 0,Marital_Status,Capital_gain,Education_num,Relationship,Sex,O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex,CF_id
0,0,Married-civ-spouse,0.568999,-0.418904,Husband,Male,0,1,0,0,0,1
1,1,Married-civ-spouse,0.585355,-0.418904,Husband,Male,0,1,0,0,0,1
2,2,Married-civ-spouse,0.623752,-0.418904,Husband,Male,0,1,0,0,0,1
3,3,Married-civ-spouse,0.637752,-0.418904,Husband,Male,0,1,0,0,0,1
4,4,Married-civ-spouse,0.660623,-0.418904,Husband,Male,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18587,18587,Widowed,2.646975,-1.972600,Other-relative,Female,1,1,1,1,1,186
18588,18588,Widowed,2.646975,-2.361024,Other-relative,Female,1,1,1,1,1,186
18589,18589,Widowed,2.646975,-2.749448,Other-relative,Female,1,1,1,1,1,186
18590,18590,Widowed,2.646975,-3.137872,Other-relative,Female,1,1,1,1,1,186


In [839]:
Feature_Columns

['Marital_Status', 'Capital_gain', 'Education_num', 'Relationship', 'Sex']

In [840]:
df_cf_2

,Unnamed: 0,Marital_Status,Capital_gain,Education_num,Relationship,Sex,O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex,CF_id
0,0,Married-civ-spouse,0.568999,-0.418904,Husband,Male,0,1,0,0,0,1
1,1,Married-civ-spouse,0.585355,-0.418904,Husband,Male,0,1,0,0,0,1
2,2,Married-civ-spouse,0.623752,-0.418904,Husband,Male,0,1,0,0,0,1
3,3,Married-civ-spouse,0.637752,-0.418904,Husband,Male,0,1,0,0,0,1
4,4,Married-civ-spouse,0.660623,-0.418904,Husband,Male,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18587,18587,Widowed,2.646975,-1.972600,Other-relative,Female,1,1,1,1,1,186
18588,18588,Widowed,2.646975,-2.361024,Other-relative,Female,1,1,1,1,1,186
18589,18589,Widowed,2.646975,-2.749448,Other-relative,Female,1,1,1,1,1,186
18590,18590,Widowed,2.646975,-3.137872,Other-relative,Female,1,1,1,1,1,186


In [841]:
# Get a df of disances of each cf compared to the reference row, i.e., the original sample (scaled)
#df_cf_distances = add_distance_columns1(Feature_Columns, df_cf_2.copy(), reference_row, feature_types)
df_cf_distances = add_distance_columns_vectorized(Feature_Columns, df_cf_2.copy(), reference_row, feature_types)
df_cf_distances

,Unnamed: 0,Marital_Status,Capital_gain,Education_num,Relationship,Sex,O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex,CF_id,L0,L1,L2
0,0,Married-civ-spouse,0.568999,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.701392,0.701392
1,1,Married-civ-spouse,0.585355,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.717748,0.717748
2,2,Married-civ-spouse,0.623752,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.756145,0.756145
3,3,Married-civ-spouse,0.637752,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.770145,0.770145
4,4,Married-civ-spouse,0.660623,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.793016,0.793016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18587,18587,Widowed,2.646975,-1.972600,Other-relative,Female,1,1,1,1,1,186,5,7.333064,3.624757
18588,18588,Widowed,2.646975,-2.361024,Other-relative,Female,1,1,1,1,1,186,5,7.721489,3.807456
18589,18589,Widowed,2.646975,-2.749448,Other-relative,Female,1,1,1,1,1,186,5,8.109913,4.019493
18590,18590,Widowed,2.646975,-3.137872,Other-relative,Female,1,1,1,1,1,186,5,8.498337,4.256486


In [842]:
k

20

In [843]:

k_smallest_cf_df_file_name = 'vector_'+model+'_adult_'+str(k)+'_smallest_'+norm+'.csv'


In [844]:
# Select the k nearest counterfactuals as per the norm
df_k_smallest,  k_smallest_indices = get_k_smallest_points(df_cf_distances, distance_col=norm, k=k)
print(df_k_smallest)


     Unnamed: 0      Marital_Status  Capital_gain  Education_num Relationship  \
0             0  Married-civ-spouse      0.568999      -0.418904      Husband   
1             1  Married-civ-spouse      0.585355      -0.418904      Husband   
2             2  Married-civ-spouse      0.623752      -0.418904      Husband   
3             3  Married-civ-spouse      0.637752      -0.418904      Husband   
4             4  Married-civ-spouse      0.660623      -0.418904      Husband   
160         160  Married-civ-spouse      0.568999      -0.807328      Husband   
20           20  Married-civ-spouse      0.568999      -0.030480      Husband   
27           27  Married-civ-spouse      0.585355      -0.030480      Husband   
168         168  Married-civ-spouse      0.585355      -0.807328      Husband   
5             5  Married-civ-spouse      0.712742      -0.418904      Husband   
34           34  Married-civ-spouse      0.623752      -0.030480      Husband   
176         176  Married-civ

In [845]:
df_fact.iloc[ref_index]

Marital_Status         Divorced
Capital_gain                  0
Education_num                 9
Relationship      Not-in-family
Sex                        Male
Name: 1, dtype: object

In [846]:
df_k_smallest

,Unnamed: 0,Marital_Status,Capital_gain,Education_num,Relationship,Sex,O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex,CF_id,L0,L1,L2
0,0,Married-civ-spouse,0.568999,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.701392,0.701392
1,1,Married-civ-spouse,0.585355,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.717748,0.717748
2,2,Married-civ-spouse,0.623752,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.756145,0.756145
3,3,Married-civ-spouse,0.637752,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.770145,0.770145
4,4,Married-civ-spouse,0.660623,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.793016,0.793016
160,160,Married-civ-spouse,0.568999,-0.807328,Husband,Male,0,1,1,0,0,3,2,1.089816,0.801763
20,20,Married-civ-spouse,0.568999,-0.030480,Husband,Male,0,1,1,0,0,2,2,1.089816,0.801763
27,27,Married-civ-spouse,0.585355,-0.030480,Husband,Male,0,1,1,0,0,2,2,1.106172,0.816110
168,168,Married-civ-spouse,0.585355,-0.807328,Husband,Male,0,1,1,0,0,3,2,1.106172,0.816110
5,5,Married-civ-spouse,0.712742,-0.418904,Husband,Male,0,1,0,0,0,1,1,0.845135,0.845135


In [847]:
df_k_smallest
k_smallest_indices
df_cf.iloc[k_smallest_indices]

,Unnamed: 0,Marital_Status,Capital_gain,Education_num,Relationship,Sex,O_Marital_Status,O_Capital_gain,O_Education_num,O_Relationship,O_Sex,CF_id
0,0,Married-civ-spouse,5060,9,Husband,Male,0,1,0,0,0,1
1,1,Married-civ-spouse,5178,9,Husband,Male,0,1,0,0,0,1
2,2,Married-civ-spouse,5455,9,Husband,Male,0,1,0,0,0,1
3,3,Married-civ-spouse,5556,9,Husband,Male,0,1,0,0,0,1
4,4,Married-civ-spouse,5721,9,Husband,Male,0,1,0,0,0,1
160,160,Married-civ-spouse,5060,8,Husband,Male,0,1,1,0,0,3
20,20,Married-civ-spouse,5060,10,Husband,Male,0,1,1,0,0,2
27,27,Married-civ-spouse,5178,10,Husband,Male,0,1,1,0,0,2
168,168,Married-civ-spouse,5178,8,Husband,Male,0,1,1,0,0,3
5,5,Married-civ-spouse,6097,9,Husband,Male,0,1,0,0,0,1


In [848]:
# Obtain the complete list of k smallest counterfactuals with all the columns
df_cf_final = df_cf.iloc[k_smallest_indices][Feature_Columns].copy()
df_cf_final

,Marital_Status,Capital_gain,Education_num,Relationship,Sex
0,Married-civ-spouse,5060,9,Husband,Male
1,Married-civ-spouse,5178,9,Husband,Male
2,Married-civ-spouse,5455,9,Husband,Male
3,Married-civ-spouse,5556,9,Husband,Male
4,Married-civ-spouse,5721,9,Husband,Male
160,Married-civ-spouse,5060,8,Husband,Male
20,Married-civ-spouse,5060,10,Husband,Male
27,Married-civ-spouse,5178,10,Husband,Male
168,Married-civ-spouse,5178,8,Husband,Male
5,Married-civ-spouse,6097,9,Husband,Male


In [849]:
# 1) Identify extra columns:
extra_cols = set(df_tmp_ref.columns) - set(df_cf_final.columns)
# e.g. {'Extra1', 'Extra2'}

# 2) Extract the SINGLE instance from df_B
#    (assuming df_B really has only one row, or you specifically want the 0th row)
row_b = df_tmp_ref.iloc[0]  # This is a pandas Series representing that row.

# 3) For each extra column, set all rows of df_A to the single value from df_B
for col in extra_cols:
    df_cf_final[col] = row_b[col]

# Now df_A has the extra columns, each row filled with the single-instance value from df_B
print(df_cf_final)

         Marital_Status  Capital_gain  Education_num Relationship   Sex  \
0    Married-civ-spouse          5060              9      Husband  Male   
1    Married-civ-spouse          5178              9      Husband  Male   
2    Married-civ-spouse          5455              9      Husband  Male   
3    Married-civ-spouse          5556              9      Husband  Male   
4    Married-civ-spouse          5721              9      Husband  Male   
160  Married-civ-spouse          5060              8      Husband  Male   
20   Married-civ-spouse          5060             10      Husband  Male   
27   Married-civ-spouse          5178             10      Husband  Male   
168  Married-civ-spouse          5178              8      Husband  Male   
5    Married-civ-spouse          6097              9      Husband  Male   
34   Married-civ-spouse          5455             10      Husband  Male   
176  Married-civ-spouse          5455              8      Husband  Male   
41   Married-civ-spouse  

In [850]:
df_cf_final

,Marital_Status,Capital_gain,Education_num,Relationship,Sex,Occupation,Native_country,Fnlwgt,Age,Race,Hours_per_week,Workclass
0,Married-civ-spouse,5060,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov
1,Married-civ-spouse,5178,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov
2,Married-civ-spouse,5455,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov
3,Married-civ-spouse,5556,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov
4,Married-civ-spouse,5721,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov
160,Married-civ-spouse,5060,8,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov
20,Married-civ-spouse,5060,10,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov
27,Married-civ-spouse,5178,10,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov
168,Married-civ-spouse,5178,8,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov
5,Married-civ-spouse,6097,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov


In [851]:
# Add distances to the counterfactual dataframe
# Specify the columns to add from df_B
columns_to_add = ['L0', 'L1', 'L2']  # Replace with actual column names

# Add them to df_A
df_cf_final[columns_to_add] = df_k_smallest[columns_to_add]

In [852]:
df_cf_final
df_cf_final.head(k)

,Marital_Status,Capital_gain,Education_num,Relationship,Sex,Occupation,Native_country,Fnlwgt,Age,Race,Hours_per_week,Workclass,L0,L1,L2
0,Married-civ-spouse,5060,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,1,0.701392,0.701392
1,Married-civ-spouse,5178,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,1,0.717748,0.717748
2,Married-civ-spouse,5455,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,1,0.756145,0.756145
3,Married-civ-spouse,5556,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,1,0.770145,0.770145
4,Married-civ-spouse,5721,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,1,0.793016,0.793016
160,Married-civ-spouse,5060,8,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,2,1.089816,0.801763
20,Married-civ-spouse,5060,10,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,2,1.089816,0.801763
27,Married-civ-spouse,5178,10,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,2,1.106172,0.816110
168,Married-civ-spouse,5178,8,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,2,1.106172,0.816110
5,Married-civ-spouse,6097,9,Husband,Male,Adm-clerical,United-States,77516,39,White,40,State-gov,1,0.845135,0.845135


In [853]:
df_cf_final.to_csv('./Counterfactuals/'+k_smallest_cf_df_file_name)

In [854]:
k_smallest_cf_df_file_name

'vector_MC3G_adult_20_smallest_L2.csv'